Process
== 
> Fill in a module description here

In [ ]:
#| default_exp process

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2
%matplotlib inline
     

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import numpy as np
from fastai.basics import *


## Import Log

Column renaming and adding of start and end events

In [ ]:
#| export
def df_preproc (df,cols=['activity'],start_marker='###start###',end_marker='###end###'):
    """
    Add event_id column, as start and end tokens for each trace
    - Copys entire row.
    - Adds special start and end marker for col in `cols`
    
    """
    # Add event log column
    df['event_id']=df.groupby('trace_id').cumcount()+1    
    # Work with numpy for performance boost
    eid_col = df.columns.to_list().index('event_id')
       
    # Get col idx
    col_idx=[df.columns.to_list().index(c) for c in cols]
        
    data= df.values
    # Add Start Events
    idx= np.where(data[:,eid_col]==1)[0] # start idx
    new = data[idx].copy()
    
    for c in col_idx: new[:,c]=start_marker
    new[:,eid_col]=0
    data = np.insert(data,idx,new, axis=0)
    # Add End Events
    idx= np.where(data[:,eid_col]==0)[0][1:] # start idx without the first
    new = data[idx-1].copy() # get data from current last idx
    for c in col_idx: new[:,c]=end_marker
    
    new[:,eid_col]=new[:,eid_col]+1
    data = np.insert(data,idx,new, axis=0)
    # take care of final last event
    last= data[-1].copy()
    for c in col_idx: last[c]=end_marker

    last[eid_col]+=1
    data = np.insert(data,len(data),last, axis=0)
    df = pd.DataFrame(data,columns=df.columns)
    
    
    return df

Imports an event log in csv format

    - rename activity and trace_id column

In [ ]:
#| export

def import_log(log_path,cols=['activity']):
    """
    Imports an event log in csv format
    - rename activity and trace_id column
    - rename anomaly column
    """
    df=pd.read_csv(log_path)
    df.rename({'name':'activity','case:concept:name':'trace_id'},axis=1,inplace=True)
    if not 'activity' in df.columns:
        df.rename({'concept:name':'activity'},axis=1,inplace=True)
    df = df_preproc(df,cols)
    df.index=df.trace_id
    df.drop('trace_id',axis=1,inplace=True,)
    return df
     

In [ ]:
log_path = '../data/logs/csv/mppn_ds/BPIC12.csv'
log = import_log(log_path)
log.head()

,event_id,resource,timestamp,activity,REG_DATE,AMOUNT_REQ
trace_id,,,,,,
173688,0,112.0,2011-09-30 22:38:44.546000+00:00,###start###,2011-10-01 00:38:44.546000+02:00,20000
173688,1,112.0,2011-09-30 22:38:44.546000+00:00,A_SUBMITTED_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,2,112.0,2011-09-30 22:38:44.880000+00:00,A_PARTLYSUBMITTED_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,3,112.0,2011-09-30 22:39:37.906000+00:00,A_PREACCEPTED_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,4,112.0,2011-09-30 22:39:38.875000+00:00,W_Completeren aanvraag_SCHEDULE,2011-10-01 00:38:44.546000+02:00,20000


## Trace Splitting
i.e. splitting in training, validation and test set

The split_traces function is used to split an event_log into training, validation and test set. Furthermore, it removes traces that are longer than a specific threshhold.

In [ ]:
#| export
def drop_long_traces(df,max_trace_len=64,event_id='event_id'):
    "Drop traces longer than `max_trace_len`"
    df=df.drop(np.unique(df[df[event_id]>max_trace_len].index))
    return df

In [ ]:
#| export
def RandomTraceSplitter(split_pct=0.2, seed=None):
    "Create function that splits `items` between train/val with `valid_pct` randomly."
    def _inner(trace_ids):
        o=np.unique(trace_ids)
        np.random.seed(seed)
        rand_idx = np.random.permutation(o)
        cut = int(split_pct * len(o))
        return L(rand_idx[cut:].tolist()),L(rand_idx[:cut].tolist())
    return _inner

In [ ]:
#| export
def split_traces(df,df_name='tmp',seed = None):
    "Split traces in training, validation and test sets"
    df=drop_long_traces(df)
    ts=RandomTraceSplitter(seed=seed)
    train,test=ts(df.index)
    ts=RandomTraceSplitter(seed=seed,split_pct=0.1)
    train,valid=ts(train)
    return train,valid,test

In [ ]:
a1,b1,c1=split_traces(log)
a2,b2,c2=split_traces(log)
test_ne(a1,a2), test_ne(b1,b2), test_ne(c1,c2);

In [ ]:
seed = 42
a1,b1,c1=split_traces(log,seed = seed)
a2,b2,c2=split_traces(log,seed = seed)
test_eq(a1,a2), test_eq(b1,b2), test_eq(c1,c2);

## PPObj
Manages the pre-processing and provides utility functiosn for date columns, cat columns and cont columns.



In [ ]:
#| export
class _TraceIloc:
    "Get/set rows by iloc and cols by name"
    def __init__(self,o): self.o = o
    def __getitem__(self, idxs):
        df = self.o.items
        if isinstance(idxs,tuple):
            rows,cols = idxs
            rows=df.index[rows]
            return self.o.new(df.loc[rows,cols])
        else:
            rows,cols = idxs,slice(None)
            rows=np.unique(df.index)[rows]
            return self.o.new(df.loc[rows])

In [ ]:
#| export
# Not so useful, should be cleaned up!

class PPObj(CollBase, GetAttr, FilteredBase):
    "Main Class for Process Prediction"
    _default,with_cont='procs',True
    def __init__(self,df,procs=None,cat_names=None,cont_names=None,date_names=None,y_names=None,splits=None,
                 ycat_names=None,ycont_names=None,inplace=False,do_setup=True):
        if not inplace: df=df.copy()
        if splits is not None: df = df.loc[sum(splits, [])] # Can drop traces
        self.event_ids=df['event_id'].values if hasattr(df,'event_id') else None

        super().__init__(df)

        self.cat_names,self.cont_names,self.date_names=(L(cat_names),L(cont_names),L(date_names))
        self.set_y_names(y_names,ycat_names,ycont_names)

        self.procs = Pipeline(procs)
        self.splits=splits
        if do_setup: self.setup()


    @property
    def y_names(self): return self.ycat_names+self.ycont_names

    def set_y_names(self,y_names,ycat_names=None,ycont_names=None):
        if ycat_names or ycont_names: store_attr('ycat_names,ycont_names')
        else:
            self.ycat_names,self.ycont_names=(L([i for i in L(y_names) if i in self.cat_names]),
                                                L([i for i in L(y_names) if i not in self.cat_names]))
    def setup(self): self.procs.setup(self)
    def subset(self, i): return self.new(self.loc[self.splits[i]]) if self.splits else self
    def __len__(self): return len(np.unique(self.items.index))
    def show(self, max_n=3, **kwargs):
        print('#traces:',len(self),'#events:',len(self.items))
        display_df(self.new(self.all_cols[:max_n]).items)
    def new(self, df):
        return type(self)(df, do_setup=False,
                          **attrdict(self, 'procs','cat_names','cont_names','ycat_names','ycont_names',
                                     'date_names'))
    def process(self): self.procs(self)
    def loc(self): return self.items.loc
    def iloc(self): return _TraceIloc(self)
    def x_names (self): return self.cat_names + self.cont_names
    def all_col_names(self): return ((self.x_names+self.y_names)).unique()
    def transform(self, cols, f, all_col=True):
        if not all_col: cols = [c for c in cols if c in self.items.columns]
        if len(cols) > 0: self[cols] = self[cols].transform(f)
    def new_empty(self): return self.new(pd.DataFrame({}, columns=self.items.columns))
    def subsets(self): return [self.subset(i) for i in range(len(self.splits))] if self.splits else L(self)
properties(PPObj,'loc','iloc','x_names','all_col_names')

def _add_prop(cls, nm):
    @property
    def f(o): return o[list(getattr(o,nm+'_names'))]
    @f.setter
    def fset(o, v): o[getattr(o,nm+'_names')] = v
    setattr(cls, nm+'s', f)
    setattr(cls, nm+'s', fset)

_add_prop(PPObj, 'cat')
_add_prop(PPObj, 'cont')
_add_prop(PPObj, 'ycat')
_add_prop(PPObj, 'ycont')
_add_prop(PPObj, 'y')
_add_prop(PPObj, 'x')
_add_prop(PPObj, 'all_col')
     

In [ ]:
ppObj=PPObj(log,cat_names=['activity'],y_names=['activity'])
ppObj.show()

#traces: 13087 #events: 288374


,activity
trace_id,
173688,###start###
173688,A_SUBMITTED_COMPLETE
173688,A_PARTLYSUBMITTED_COMPLETE


We can define various pre-processing functions that are executed, when `PPOBj` is instantiated. `PPProc` is the base class for a pre-processing function. It ensures, that setup of a pre-processing function is performed using the training set, and than it is applied to the validation and test set, with the same parameters.

In [ ]:
#| export
class PPProc(InplaceTransform):
    "Base class to write a non-lazy tabular processor for dataframes"
    def setup(self, items=None, train_setup=False): #TODO: properly deal with train_setup
        super().setup(getattr(items,'train',items), train_setup=False)
        #super().setup(items, train_setup=False)

        # Procs are called as soon as data is available
        return self(items.items if isinstance(items,Datasets) else items)

    @property
    def name(self): return f"{super().name} -- {getattr(self,'__stored_args__',{})}"

### Categorization
i.e ordinal encoding

Implementation of ordinal or integer encoding. Adds NA values for unknown data. Implementation is pretty much taken from fastai.

In [ ]:
#| export
def _apply_cats (voc, add, c):
    if not is_categorical_dtype(c):
        return pd.Categorical(c, categories=voc[c.name][add:]).codes+add
    return c.cat.codes+add #if is_categorical_dtype(c) else c.map(voc[c.name].o2i)

In [ ]:
#| export
class Categorify(PPProc):
    "Transform the categorical variables to something similar to `pd.Categorical`"
    order = 2
    def setups(self, to):
        store_attr(classes={n:CategoryMap(to.items.loc[:,n], add_na=True) for n in to.cat_names}, but='to')
    def encodes(self, to):
        to.transform(to.cat_names, partial(_apply_cats, self.classes, 1))
    def __getitem__(self,k): return self.classes[k]

In [ ]:
log=import_log('../data/logs/csv/dapnn_ds/PDC2021_ground_truth/pdc2021_000000.csv.gz')
traces=split_traces(log)[0][:100]
splits=traces[:60],traces[60:80],traces[80:100]
o=PPObj(log,None,cat_names='activity',splits=splits)

In [ ]:
m=CategoryMap(o.items.loc[:,'activity'])
len(m)

50

In [ ]:
cat=Categorify()
cat.setup(o)
len(cat['activity'])

48

In [ ]:
df = pd.DataFrame({'a':[0,1,2,0,2]})
to = PPObj(df, Categorify, 'a')
to.show()

#traces: 5 #events: 5


,a
0,1
1,2
2,3


In [ ]:
log=import_log('../data/logs/csv/dapnn_ds/binet_logs/bpic12-0.3-1.csv.gz')
o=PPObj(log,Categorify,'activity')
o.show()

#traces: 13087 #events: 289892


,activity
trace_id,
173688,2
173688,12
173688,9


### Fill Missing
for continuous values

A pre-processing function that deals with missing data in continuous attributes. Missing data can be replaced with the median, mean or a constant value. Additionaly, we can create another boolean column that indicates, which rows were missing.  Implementation is pretty much taken from fastai.

In [ ]:
#| export
class FillStrategy:
    "Namespace containing the various filling strategies."
    def median  (c,fill): return c.median()
    def constant(c,fill): return fill
    def mode    (c,fill): return c.dropna().value_counts().idxmax()

In [ ]:
#| export
class FillMissing(PPProc):
    order=1
    "Fill the missing values in continuous columns."
    def __init__(self, fill_strategy=FillStrategy.median, add_col=True, fill_vals=None):
        if fill_vals is None: fill_vals = defaultdict(int)
        store_attr()

    def setups(self, dsets):
        missing = pd.isnull(dsets.conts).any()
        store_attr(but='to', na_dict={n:self.fill_strategy(dsets[n], self.fill_vals[n])
                            for n in missing[missing].keys()})
        self.fill_strategy = self.fill_strategy.__name__

    def encodes(self, to):
        missing = pd.isnull(to.conts)
        for n in missing.any()[missing.any()].keys():
            assert n in self.na_dict, f"nan values in `{n}` but not in setup training set"
        for n in self.na_dict.keys():
            to[n].fillna(self.na_dict[n], inplace=True)
            if self.add_col:
                to.loc[:,n+'_na'] = missing[n]
                if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')

In [ ]:
fill = FillMissing() 
df = pd.DataFrame({'a':[0,1,np.nan,1,2,3,4], 'b': [0,1,2,3,4,5,6]})
to = PPObj(df, fill, cont_names=['a', 'b'])
to.show()

#traces: 7 #events: 7


,a_na,a,b
0,False,0.0,0
1,False,1.0,1
2,True,1.5,2


### Z-score

In [ ]:
#| export
class Normalize(PPProc):
    "Normalize with z-score"
    order = 3
    def setups(self, to):
        store_attr(but='to', means=dict(getattr(to, 'train', to).conts.mean()),
                   stds=dict(getattr(to, 'train', to).conts.std(ddof=0)+1e-7))
        return self(to)

    def encodes(self, to): to.conts = (to.conts-self.means) / self.stds
    def decodes(self, to): to.conts = (to.conts*self.stds ) + self.means

In [ ]:
df = pd.DataFrame({'a':[0,1,9,3,4]})
to = PPObj(df, Normalize(), cont_names='a')
to.show()

#traces: 5 #events: 5


,a
0,-1.429409
1,-1.327783
2,-0.514775


### Date conversion

Encodes a date column. Supports multiple information by using pandas date functions. This implementation is also based on the fastai but also supports relative duration from the first event of a case.

In [ ]:
#| export
def _make_date(df, date_field):
    "Make sure `df[date_field]` is of the right date type."
    field_dtype = df[date_field].dtype
    if isinstance(field_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        field_dtype = np.datetime64
    if not np.issubdtype(field_dtype, np.datetime64):
        df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True,utc=True)

In [ ]:
df = pd.DataFrame({'fu': ['2019-12-04', '2019-11-29', '2019-11-15', '2019-10-24']})
_make_date(df, 'fu')
df.dtypes

fu    datetime64[ns, UTC]
dtype: object

In [ ]:
#| export
def _secSinceSunNoon(datTimStr):
    dt = pd.to_datetime(datTimStr).dt
    return (dt.dayofweek-1)*24*3600+ dt.hour * 3600 + dt.minute * 60 + dt.second

In [ ]:
#| export
def _secSinceNoon(datTimStr):
    dt = pd.to_datetime(datTimStr).dt
    return dt.hour * 3600 + dt.minute * 60 + dt.second

In [ ]:
#| export
Base_Date_Encodings=['Year', 'Month', 'Day', 'Dayofweek', 'Dayofyear','Elapsed']

In [ ]:
#| export
def encode_date(df, field_name,unit=1e9,date_encodings=Base_Date_Encodings):
    "Helper function that adds columns relevant to a date in the column `field_name` of `df`."
    _make_date(df, field_name)
    field = df[field_name]
    prefix =  re.sub('[Dd]ate$', '', field_name+"_")
    attr = ['Year', 'Month', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr:
        if n in date_encodings: df[prefix + n] = getattr(field.dt, n.lower())
    # Pandas removed `dt.week` in v1.1.10

    if 'secSinceSunNoon' in date_encodings:
        df[prefix+'secSinceSunNoon']=_secSinceSunNoon(field)
    if 'secSinceNoon' in date_encodings:
        df[prefix+'secSinceNoon']=_secSinceNoon(field)
    if 'Week' in date_encodings:
        week = field.dt.isocalendar().week if hasattr(field.dt, 'isocalendar') else field.dt.week
        df.insert(3, prefix+'Week', week)
    mask = ~field.isna()
    elapsed = pd.Series(np.where(mask,field.values.astype(np.int64) // unit,None).astype(float),index=field.index)

    if 'Relative_elapsed' in date_encodings:
        df[prefix+'Relative_elapsed']=elapsed-elapsed.groupby(elapsed.index).transform('min')

    # required to decode!
    if 'Elapsed' in date_encodings: df[prefix+'Elapsed']=elapsed

    df.drop(field_name, axis=1, inplace=True)
    return [],[prefix+i for i in date_encodings]

In [ ]:
df = pd.DataFrame({'fu': ['2019-12-04', '2019-11-29', '2019-11-15', '2019-10-24']})
encode_date(df,'fu')
df

,fu_Year,fu_Month,fu_Day,fu_Dayofweek,fu_Dayofyear,fu_Elapsed
0,2019,12,4,2,338,1.575418e+09
1,2019,11,29,4,333,1.574986e+09
2,2019,11,15,4,319,1.573776e+09
3,2019,10,24,3,297,1.571875e+09


In [ ]:
#| export
def decode_date(df, field_name,unit=1e9,date_encodings=Base_Date_Encodings):
    df[field_name]=(df[field_name+'_'+'Elapsed'] * unit).astype('datetime64[ns, UTC]')
    for c in date_encodings: del df[field_name+'_'+c]

In [ ]:
decode_date(df,'fu')
df

,fu
0,2019-12-04 00:00:00+00:00
1,2019-11-29 00:00:00+00:00
2,2019-11-15 00:00:00+00:00
3,2019-10-24 00:00:00+00:00


In [ ]:
#| export
class Datetify(PPProc):
    "Encode dates, "
    order = 0

    def __init__(self, date_encodings=['Relative_elapsed']): self.date_encodings=listify(date_encodings)

    def encodes(self, o):
        for i in o.date_names:
            cat,cont=encode_date(o.items,i,date_encodings=self.date_encodings)
            o.cont_names+=cont
            o.cat_names+=cat
# Todo: Add decoding

In [ ]:
df = pd.DataFrame({'fu': ['2019-10-04', '2019-10-09', '2019-10-15', '2019-10-24']},index=[1,1,1,1])
o = PPObj(df,Datetify(date_encodings=['secSinceSunNoon','secSinceNoon','Relative_elapsed']),date_names='fu')
o.xs

,fu_secSinceSunNoon,fu_secSinceNoon,fu_Relative_elapsed
1,259200,0,0.0
1,86400,0,432000.0
1,0,0,950400.0
1,172800,0,1728000.0


In [ ]:
df = pd.DataFrame({'fu': ['2019-10-04', '2019-10-09', '2019-10-15', '2019-10-24']},index=[1,1,1,1])
o = PPObj(df,[Datetify(date_encodings=['secSinceSunNoon','secSinceNoon','Relative_elapsed']),Normalize],date_names='fu')
o.xs

,fu_secSinceSunNoon,fu_secSinceNoon,fu_Relative_elapsed
1,-1.341627,0.0,-1.208083
1,-1.341645,0.0,-1.208082
1,-1.341655,0.0,-1.208080
1,-1.341636,0.0,-1.208079


In [ ]:
o.procs.means

{'fu_secSinceSunNoon': 129600.0,
 'fu_secSinceNoon': 0.0,
 'fu_Relative_elapsed': 777600.0}

### MinMax Scaling
Calculates the MinMax scaling from a column.

In [ ]:
#| export
class MinMax(PPProc):
    order=3

    def setups(self, o):
        store_attr(mins=o.xs.min(),
                   maxs=o.xs.max())

    def encodes(self, o):
        cols=[i+'_minmax' for i in o.x_names]
        o[cols] = o.xs.astype(float)
        o[cols] = ((o.xs-self.mins) /(self.maxs-self.mins))
        o[cols] = o.xs.astype(float)
        o.cont_names=L(cols)
        o.cat_names=L()

In [ ]:
path="../data/logs/csv/mppn_ds/BPIC12_O.csv"
log=import_log(path)
cat_names,cont_names,date_names=['activity', 'resource'], ['AMOUNT_REQ'], ['timestamp']
o=PPObj(log,[Categorify,Datetify,FillMissing,MinMax],
                     cat_names=cat_names,date_names=date_names,cont_names=cont_names,
                     y_names=['activity','resource','timestamp_Relative_elapsed'])

In [ ]:
o.xs.dtypes

activity_minmax                      float64
resource_minmax                      float64
AMOUNT_REQ_minmax                    float64
timestamp_Relative_elapsed_minmax    float64
dtype: object

### One HoT Encoding
Calculates the one-hot encoding of a column. It is required to first apply categorization on the same column, to deal with missing values.

In [ ]:
#| export
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#| export
class OneHot(PPProc):
    "Transform the categorical variables to one-hot. Requires Categorify to deal with unseen data."
    order = 3

    def encodes(self, o):
        new_cats=[]
        for c in o.cat_names:
            categories=[range(len(o.procs.categorify[c]))]
            x=o[c].to_numpy()
            ohe = OneHotEncoder(categories=categories)
            enc=ohe.fit_transform(x.reshape(-1, 1)).toarray()
            for i in range(enc.shape[1]):
                new_cat=f'{c}_{i}'
                o.items.loc[:,new_cat]=enc[:,i]
                new_cats.append(new_cat)
        o.cat_names=L(new_cats)

In [ ]:
event_df=import_log('../data/logs/csv/dapnn_ds/PDC2021_training/pdc2021_0000000.csv.gz')

In [ ]:
%%time
o=PPObj(event_df,[Categorify(),OneHot()],cat_names=['activity'])

## Window Generation
Here, we cover the sliding window generation

In [ ]:
#| export
def _shift_columns (a,ws=3): return np.dstack(list(reversed([np.roll(a,i) for i in range(0,ws)])))[0]

def windows_fast(df,event_ids,ws=5,pad=None):
    max_trace_len=int(event_ids.max())+1
    trace_start = np.where(event_ids == 0)[0]
    trace_len=[trace_start[i]-trace_start[i-1] for i in range(1,len(trace_start))]+[len(df)-trace_start[-1]]
    idx=[range(trace_start[i]+(i+1)
               *(ws-1),trace_start[i]+trace_len[i]+(i+1)*(ws-1)-1) for i in range(len(trace_start))]
    idx=np.array([y for x in idx for y in x])
    trace_start = np.repeat(trace_start, ws-1)
    tmp=np.stack([_shift_columns(np.insert(np.array(df[i]), trace_start, 0, axis=0),ws=ws) for i in list(df)]) 
    tmp=np.rollaxis(tmp,1) 
    res=tmp[idx]
    if pad: res=np.pad(res,((0,0),(0,0),(pad-ws,0))) 
    
    return res,np.where(event_ids != 0)[0]


In [ ]:
event_df=import_log('../data/logs/csv/dapnn_ds/PDC2020_ground_truth/pdc_2020_0000000.csv.gz')

In [ ]:
o=PPObj(event_df,Categorify(),cat_names=['activity'],y_names='activity')
#o=o.iloc[0]
len(o)

In [ ]:
ws,idx=windows_fast(o.xs,o.event_ids,ws=5)
ws,ws.shape

## Data Loader
The prefixes are converted to a `pytorch.Dataset` and than to a `DataLoader`
A batch is than represented as a tuple of the form `(x cat. attr,x cont. attr, y cat. attr., y cont attr.)`. Also, categorical attributes are converted to a long tensor and continous attributes to a float tensor.

If a dimensions of the batch is empty - e.g. the model does not use categorical input attributes - it is removed from the tuple. 

In [ ]:
o=PPObj(event_df,Categorify(),cat_names=['activity'],y_names='activity')
ws,idx=windows_fast(o.xs,o.event_ids,ws=10)
ws,ws.shape

In [ ]:
#| export
class PPDset(torch.utils.data.Dataset):
    "some doc"
    def __init__(self, inp):
        store_attr('inp')

    def __len__(self): return len(self.inp[0])

    def __getitem__(self, idx):
        xs=tuple([i[idx]for i in self.inp[:-1]])
        ys=tuple([i[idx]for i in self.inp[-1]])
        #if len(ys)==1: ys=ys[0]
        return (*xs,ys)

In [ ]:
#| exporti
@delegates(TfmdDL)
def get_dls(ppo:PPObj,windows=windows_fast,outcome=False,event_id='event_id',bs=64,**kwargs):
    "some doc"
    ds=[]
    for s in ppo.subsets():
        wds,idx=windows(s.xs,s.event_ids)

        if not outcome: y=s.ys.iloc[idx]
        else: y=s.ys.groupby(s.items.index).transform('last').iloc[idx]
        ycats=tensor(y[s.ycat_names].values).long()
        yconts=tensor(y[s.ycont_names].values).float()
        xconts=tensor(wds[:,len(s.cat_names):]).float()
        xcats=tensor(wds[:,:len(s.cat_names)]).long()
        xs=tuple([i for i in [xcats,xconts] if i.shape[1]>0])
        ys=tuple([ycats[:,i] for i in range(ycats.shape[1])])+tuple([yconts[:,i] for i in range(yconts.shape[1])])
        ds.append(PPDset((*xs,ys)))
    return DataLoaders.from_dsets(*ds,bs=bs,device=torch.device('cuda'),**kwargs)
PPObj.get_dls= get_dls

In [ ]:
dls=o.get_dls()
xb,yb=dls.one_batch()
xb.shape,y[0].shape

### Integration Samples
This section shows, how the PPObj can be used to create a DataLoader for pedictive process analytics:

Next event prediction:  
X: 'activity'   
Y: 'activity'

In [ ]:
log=import_log('../data/logs/csv/dapnn_ds/PDC2020_ground_truth/pdc_2020_0000001.csv.gz')
o=PPObj(log,Categorify(),cat_names=['activity'],y_names='activity',splits=split_traces(event_df))
o

In [ ]:
dls=o.get_dls(windows=partial(windows_fast,ws=2))
xb,y=dls.one_batch()
xb.shape,y[0].shape


## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()